# Cosmology
Here are how to use the MRNN to do some cosmological predictions. To specify, the intrinsic luminosity of SNe.  

In [1]:
import os
import glob
import yaml
import keras
import astropy
import numpy as np
import pandas as pd
import keras.backend as K
import astropy.units as u
import astropy.constants as c
from astropy.time import Time
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from keras.models import Sequential,Model
from dust_extinction.averages import GCC09_MWAvg
from sklearn.model_selection import train_test_split
from dust_extinction.parameter_averages import CCM89,F99

Using TensorFlow backend.


## The B-band Magnitude Measure
I got a rough B-band transmission, hope it is accurate enough.  
And when you are using this B-band calculator, please check the wavelength sampling. In our spectra which has 2000 pixels between 2000 and 10000 Angstrom wavelength, the delta frequency is the default number.  

In [2]:
def BFilterMagCalculator(spec,delta_frequency=5.9958492*10**11*u.Hz):
    #shortwave=2000
    #longwave=10000
    spec=pd.DataFrame(spec,columns=['wave','flux'])
    #flux=spec[spec['wave']>shortwave]
    #flux=flux[flux['wave']<longwave]
    wave=np.array(spec['wave'])
    flux=np.array(spec['flux'])
    Bfilter=np.array([0.0,0.0,0.030,0.134,0.567,
                      0.920,0.978,1.000,0.978,
                      0.935,0.853,0.740,0.640,
                      0.536,0.424,0.325,0.235,
                      0.150,0.095,0.043,0.009,0.0,0.0])
    Bwave=np.array([2000,3600,3700,3800,3900,
                    4000,4100,4200,4300,
                    4400,4500,4600,4700,
                    4800,4900,5000,5100,
                    5200,5300,5400,5500,5600,10000])
    Bfunc=interp1d(Bwave,Bfilter)#The Johnson B filter
    flux=flux*Bfunc(wave)
    flux=flux/(1.1964952*10**40)#The spherical luminosity to the flux in a square centimeter at 10 parsec away. 
    delta_lambda=(wave*u.AA)**2*delta_frequency/c.c
    delta_lambda=delta_lambda.to(unit=u.AA)
    flux=sum(flux*delta_lambda.value)
    #print(flux*(1.1964952*10**40))
    #print(flux)
    mag=-2.5*np.log10(flux/6.00069/10**-6)+0.03# Here it is the flux divided by Vega's apparent flux in B band by HST STIS measurement. 
    return mag

In [ ]:
X=np.load('DataSet/X.npy')
Y=np.zeros(X.shape[0])
wave=np.load('DataSet/wave.npy')
X=X/np.average(X[:,167:269])
Y=Y-Yaverage/Ystd
Y=np.tanh(Y)/2+0.5
Y=pd.DataFrame(Y,columns=[str(i)+'_'+str(j) for i in range(6,29) for j in range(1,5)])

In [ ]:
data=[]
for i in [1,2,3,4,5,6,7,8,9,13,14,15,16,17,18,19,20,21]:
    data.append(pd.read_csv('csvs/'+str(i)+'.csv'))

In [ ]:
specdirs=['Spec/'+str(i)+'/*' for i in [1,2,3,4,5,6,7,8,9,13,14,15,16,17,18,19,20,21]]
specs=[]
speccounts=[]
for i in specdirs:
    specsmall=[]
    speccountsmall=[]
    for j in glob.glob(i):
        #if int(j.split('/')[2].split('.txt')[0])
        k=np.genfromtxt(j)
        specsmall.append(k)
        speccountsmall.append(int(j.split('/')[2].split('.txt')[0]))
    print(i)
    specs.append(specsmall)
    speccounts.append(speccountsmall)

In [ ]:
X=[]
Y=[]
Mags=[]
for i in range(len(specs)):
    for j in range(len(specs[i])):
        spdata=specs[i][j]
        bmag=BFilterMagCalculator(spdata).copy()
        if bmag>-17.5:continue
        spdata=Normalizer(spdata)
        if np.max(spdata[:,1])>30:continue
        Mags.append(bmag)
        X.append(spdata[:,1])
        Y.append(np.array(data[i].iloc[speccounts[i][j],3:]))
X=np.array(X)
X=X.reshape(X.shape[0],2000,1)
Y=np.array(Y)

Y=(Y-Yaverage)/Ystd
Y=np.tanh(Y)/2+0.5
Y=pd.DataFrame(Y,columns=[str(i)+'_'+str(j) for i in range(6,29) for j in range(1,5)])
Mags=np.array(Mags)

magaver=-19
magstd=1

Ymag=np.tanh((Mags-magaver)/magstd)/2+0.5
Ymag=pd.DataFrame({'mag':Ymag})